In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import xarray as xr
import seaborn as sns

from sdm_eurec4a.visulization import ncols_nrows_from_N, label_from_attrs

In [ ]:
L_v = 2.5e6  # J/kg
R_v = 461.5  # J/kg/K
T0 = 273.15  # K
p0 = 1013.25  # hPa
rho_water = 1000  # kg/m^3


class Thermodyanmics:

    def __init__(self, T0: float, p0: float):

        self.T0 = T0  # K
        self.p0 = p0  # hPa
        self.set_constants()

    def set_constants(self):

        self._L_v = 2.5e6  # J/kg
        self._R_v = 461.5  # J/kg/K
        self._rho_water = 1000

        # ============================
        # constants for air
        # =============================

        self._rhow = 0.998e3
        self._rhoa = 1.2
        self._eta = 1.85e-5
        self._g = 9.81
        self._nu = self._eta / self._rhoa
        self._Dv = 2.2e-5
        self._Sc = self._nu / self._Dv
        self._gamma = 73e-3
        self._Mmol = 2.99e-26
        self._ns = 5.7e23
        self._Coo = 0.26
        self._Cgamma = 0.1
        self._lgamma = np.sqrt(self._gamma / (self._rhow * self._g))
        self._A = (3 * self._Mmol / (4 * np.pi * self._rhow)) ** (1 / 3)

        self._Re = 2.44
        self._fv = 0.78 + 0.308 * self._Sc ** (1 / 3) * self._Re ** (1 / 2)

    def constant_florian(self, relative_humidity: float) -> float:
        return 2 * self._fv * self._Dv * self._Mmol * self._ns / self._rhow * (1 - relative_humidity)

    def saturation_vapour_pressure(self, T: float):
        """saturation vapor pressure over water"""
        A_w = 2.53e11  # Pa
        B_w = 5420  # K

        return A_w * np.exp(-B_w / T)

    def K(self, T: float) -> float:
        return 4.1868e-3 * (5.60 + 0.017 * (T - self.T0))

    def F(self, T: float, p: float) -> float:
        F_k_l = self._L_v**2 / (self._R_v * T**2 * self.K(T))
        D_v = 2.11e-5 * (T / self.T0) ** 1.94 * (p / self.p0)
        F_d_l = self._R_v / (D_v * self.saturation_vapour_pressure(T))

        F = 1 / (F_d_l + F_k_l) * 1 / self._rho_water
        return F

    def constant_nils(self, T: float, p: float, relative_humidity: float) -> float:

        return 2 * self.F(T=T, p=p) * (1 - relative_humidity)

    from typing import Callable

    def terminal_velocity(self, r: xr.DataArray) -> xr.DataArray:

        k1 = 1.2e6 * 1e-2  # m^-1 s^-1
        eq739 = k1 * r**2
        eq739 = eq739.where(r <= 0.03e-3, 0)

        k2 = 8000  # s^-1
        eq740 = k2 * r
        eq740 = eq740.where((r > 0.03e-3) & (r <= 0.6e-3), 0)

        k3 = 2010 * np.sqrt(1e-2)  # m^2 s^-1
        eq743 = k3 * np.sqrt(r)
        eq743 = eq743.where(r > 0.6e-3, 0)

        return eq739 + eq740 + eq743

    from typing import Literal

    def evapoartion_fraction_func(
        self,
        radius: xr.DataArray,
        height: float,
        method: Literal["nils", "florian"] = "nils",
        kwargs: dict = dict(),
    ) -> xr.DataArray:
        """radius needs to be given in m"""

        if method == "nils":
            c = self.constant_nils(
                T=kwargs["T"], p=kwargs["p"], relative_humidity=kwargs["relative_humidity"]
            )
        elif method == "florian":
            c = self.constant_florian(relative_humidity=kwargs["relative_humidity"])
        else:
            raise ValueError("Method not implemented")
        C = c * height / self.terminal_velocity(radius)

        result = -1 + (1 - C * radius ** (-2)) ** (3 / 2)
        return result

    def plot_terminal_velocity(self, r: xr.DataArray):
        fig, ax = plt.subplots()
        v_t = self.terminal_velocity(r)
        ax.plot(r, v_t)
        ax.set_xscale("log")
        ax.set_yscale("log")
        return ax


def saturation_vapour_pressure(T: float):
    """saturation vapor pressure over water"""
    A_w = 2.53e11  # Pa
    B_w = 5420  # K

    return A_w * np.exp(-B_w / T)

In [ ]:
L_v = 2.5e6  # J/kg
R_v = 461.5  # J/kg/K
rho_water = 1000

T0 = 273.15  # K
p0 = 1013.25  # hPa

T = np.linspace(260, 330, 30)
p = np.geomspace(1000, 100, 30)
velocity = np.geomspace(0.01, 10, 10)

ds_c = xr.Dataset()
ds_c["temperature"] = T
ds_c["pressure"] = p
ds_c["velocity"] = velocity

ds_c["3d_temperature"] = ds_c["temperature"].expand_dims(
    pressure=ds_c["pressure"], velocity=ds_c["velocity"]
)
ds_c["3d_pressure"] = ds_c["pressure"].expand_dims(
    temperature=ds_c["temperature"], velocity=ds_c["velocity"]
)
ds_c["3d_velocity"] = ds_c["velocity"].expand_dims(
    temperature=ds_c["temperature"], pressure=ds_c["pressure"]
)
ds_c = ds_c.transpose("temperature", "pressure", "velocity")

RH = 0.8
height = 1000

# K = lambda T :4.1868e-3 * (5.60 + 0.017 * (T - T0))
# F_k_l = lambda T :  L_v**2 / (R_v * T**2 * K(T))
# D_v = lambda T, p :  2.11e-5 * (T / T0)**1.94 * (p / p0)
# F_d_l = lambda T, p : R_v / (D_v(T, p) * saturation_vapour_pressure(T))

# F = 1/ (F_d_l(T, p) + F_k_l(T)) * 1/ rho_water

# F = 1/ (F_d_l(ds_c['temperature'], ds_c['pressure']) + F_k_l(ds_c['temperature'])) * 1/ rho_water
# ds_c['c'] = 2 * F * (1 - RH) * height / ds_c['velocity']

td = Thermodyanmics(T0=273.15, p0=1013.25)
ds_c["c_nils"] = (
    td.constant_nils(T=ds_c["temperature"], p=ds_c["pressure"], relative_humidity=RH)
    * height
    / ds_c["velocity"]
)
ds_c["c_florian"] = td.constant_florian(relative_humidity=RH) * height / ds_c["velocity"]

# fig, axs = plt.subplots(nrows = len(ds_c['c_nils'].dims), ncols = 3, figsize = (15, 15))

# da = ds_c['c_nils']
# for i, dim in enumerate(da.dims):
#     dim = dim
#     da.min(dim = dim).plot(ax = axs[i, 0])
#     axs[i, 0].set_title(f"min {dim}")
#     da.mean(dim = dim).plot(ax = axs[i, 1])
#     axs[i, 1].set_title(f"mean {dim}")
#     da.max(dim = dim).plot(ax = axs[i, 2])
#     axs[i, 2].set_title(f"max {dim}")

# for _ax in axs.flatten():
#     _ax.set_xscale('log')
#     _ax.set_yscale('log')

In [ ]:
vars = ["temperature", "velocity", "pressure"]


reference = dict(temperature=273.15, velocity=1, pressure=1013.25)

# all combinations are
import itertools

combinations = list(itertools.combinations(vars, 2))
nrows, ncols = 1, len(vars) + 1
widths = [1] * (ncols - 1) + [0.1]


cmap = sns.color_palette("rocket_r", as_cmap=True)
norm = mcolors.LogNorm(vmin=ds_c["c_nils"].min(), vmax=ds_c["c_nils"].max())

fig, axs = plt.subplots(nrows=nrows, ncols=ncols, width_ratios=widths, figsize=(15, 5))

for current, ax in zip(combinations, axs.flatten()):
    left = list(set(vars) - set(current))[0]
    x_var, y_var = current

    x = ds_c["3d_" + x_var].sel({left: reference[left]}, method="nearest").transpose(x_var, y_var)
    y = ds_c["3d_" + y_var].sel({left: reference[left]}, method="nearest").transpose(x_var, y_var)
    c = ds_c["c_nils"].sel({left: reference[left]}, method="nearest").transpose(x_var, y_var)

    ax.scatter(x, y, c=c, cmap=cmap, norm=norm)
    # ax.set_xscale('log')
    # ax.set_yscale('log')
    ax.set_xlabel(label_from_attrs(x))
    ax.set_ylabel(label_from_attrs(y))

fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), cax=axs[-1])
fig.tight_layout()

In [ ]:
vars = ["temperature", "velocity", "pressure"]


reference = dict(temperature=[270, 290, 300], velocity=[0, 1, 2], pressure=[0, 850, 1013.25])

# all combinations are
import itertools

combinations = list(itertools.combinations(vars, 2))
nrows, ncols = 3, len(vars)

cmap = sns.color_palette("rocket_r", as_cmap=True)
norm = mcolors.LogNorm(vmin=ds_c["c_nils"].min(), vmax=ds_c["c_nils"].max())

fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(7, 7))


for current, i in zip(combinations, range(nrows)):
    left = list(set(vars) - set(current))[0]
    x_var, y_var = current

    reference_values = reference[left]

    for reference_value, j in zip(reference_values, range(ncols)):
        ax = axs[j, i]

        da_left = ds_c[left].sel({left: reference_value}, method="nearest")

        x = ds_c["3d_" + x_var].sel({left: da_left}).transpose(x_var, y_var, ...)
        y = ds_c["3d_" + y_var].sel({left: da_left}).transpose(x_var, y_var, ...)
        c = ds_c["c_nils"].sel({left: da_left}).transpose(x_var, y_var, ...)

        sc = ax.scatter(x, y, c=c, cmap=cmap, norm=norm, marker=".")
        if x_var != "temperature":
            ax.set_xscale("log")
        if y_var != "temperature":
            ax.set_yscale("log")
        # ax.set_xscale('log')
        # ax.set_yscale('log')
        ax.set_xlabel(label_from_attrs(x))
        ax.set_ylabel(label_from_attrs(y))
        ax.set_title(f"{left} = {da_left.values:.0f}")
fig.tight_layout()
cax = fig.add_axes([1.0, 0.05, 0.02, 0.9])
fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cax)

In [ ]:
ds = ds_c.sel(temperature=slice(290, 310), pressure=slice(1000, 850), velocity=slice(0, 10))


combinations = np.array(list(itertools.combinations(vars, 2)))

nrows, ncols = combinations.T.shape

fig, axs = plt.subplots(nrows=1, ncols=ncols, figsize=(15, 5))


for var, ax in zip(vars, axs.flatten()):
    x = ds["3d_" + var]
    y = ds["c_nils"]

    ax.scatter(x, y)
    # ax.set_xscale('log')
    ax.set_yscale("log")
    ax.set_xlabel(label_from_attrs(x))
    ax.set_ylabel(label_from_attrs(y))

fig.tight_layout()

In [ ]:
r = np.geomspace(50e-6, 3e-3, 100)
m = np.geomspace(1e-3, 1e-3, 100)

ds = xr.Dataset()
T = np.linspace(220, 313, 10)
p = np.geomspace(1000, 100, 10)
velocity = np.geomspace(0.01, 10, 10)

ds["r"] = xr.DataArray(r, dims=("r",))
ds["temperature"] = T
ds["pressure"] = p


RH = 0.85
height = 800

# ds['c'] = ds_c['c']


td = Thermodyanmics(T0=273.15, p0=1013.25)
ds["velocity"] = td.terminal_velocity(ds["r"])
ds["c"] = (
    td.constant_nils(T=ds["temperature"], p=ds["pressure"], relative_humidity=RH)
    * height
    / ds["velocity"]
)
ds["c_florian"] = td.constant_florian(relative_humidity=RH) * height / ds["velocity"]

ds["ef"] = td.evapoartion_fraction_func(
    radius=ds["r"],
    height=height,
    method="nils",
    kwargs={"T": ds["temperature"], "p": ds["pressure"], "relative_humidity": RH},
)
ds["ef"] = ds["ef"].where(ds["ef"] > -1, -1)

ds["ef_florian"] = td.evapoartion_fraction_func(
    radius=ds["r"], height=height, method="florian", kwargs={"relative_humidity": RH}
)
ds["ef_florian"] = ds["ef_florian"].where(ds["ef_florian"] > -1, -1)

In [ ]:
tu = (
    dict(temperature=300, pressure=1000, method="nearest"),
    dict(temperature=240, pressure=1000, method="nearest"),
)
for t in tu:
    style = dict(color="orange", linestyle="-", alpha=1)

    if t["temperature"] != 300:
        style["color"] = "red"
    if t["pressure"] == 1000:
        style["linestyle"] = "-"
    elif t["pressure"] == 500:
        style["linestyle"] = "--"
        style["color"] = "green"

    data = -100 * ds["ef"].sel(**t)
    label = f"{data['temperature'].values:.0f}K, {data['pressure'].values:.0f}hPa"
    plt.plot(data["r"], data, label=label, **style)

plt.plot(
    ds["r"],
    -100 * ds["ef_florian"],
    label="florian",
    color="k",
)

plt.plot(
    ds["r"],
    100 * 3 / 2 * ds["c_florian"] * ds["r"] ** (-2),
    label=r"$c_{flo} r^{-2}$",
    color="blue",
)

c_nils = ds["c"].sel(temperature=240, pressure=1000, method="nearest")
plt.plot(
    ds["r"],
    100 * 3 / 2 * c_nils * ds["r"] ** (-2),
    label=r"$c_{nils} r^{-2}$",
    color="purple",
    linestyle="-",
)
plt.plot(
    ds["r"],
    100
    * (
        3 / 2 * c_nils * ds["r"] ** (-2) - 3 / 4 * c_nils**2 * ds["r"] ** (-4)
    ),  #  + 15/16 * c_nils ** 3 * ds['r'] ** (-6)),
    label=r"$c_{nils} r^{-2} + r^{-4}$",
    color="green",
    linestyle="-",
)


plt.legend()
plt.xscale("log")
plt.yscale("log")
# plt.ylim(1.1e-1, 1.05e2)
# plt.xlim(50e-6, 4e-3)

In [ ]:
cs = np.geomspace(1e-12, 5e-2, 20)


fig, axs = plt.subplots(figsize=(15, 15), **ncols_nrows_from_N(len(cs)), sharex=True, sharey=True)

for i, c in enumerate(cs):
    ax = axs.flatten()[i]
    ax_t = ax.twinx()
    approximations = [
        (3 / 2 * c * ds["r"] ** (-2)),
        ((3 * 5) / (2 * 4) * c**2 * ds["r"] ** (-4)),
        ((3 * 5 * 7) / (2 * 4 * 6) * c**3 * ds["r"] ** (-6)),
    ]
    ax.plot(
        ds["r"],
        (3 / 2 * c * ds["r"] ** (-2)),
        label="c r(-2)",
    )
    ax.plot(
        ds["r"],
        ((3 * 5) / (2 * 4) * c**2 * ds["r"] ** (-4)),
        label="c(2) r(-4)",
    )
    ax.plot(
        ds["r"],
        ((3 * 5 * 7) / (2 * 4 * 6) * c**3 * ds["r"] ** (-6)),
        label="c(3) r(-6)",
    )

    ax.set_title(f"c = {c:.2e}")


axs.flatten()[0].legend()
ax.set_xscale("log")
ax.set_yscale("log")

fig.supxlabel("radius [$m$]")
fig.supylabel("Approximation terms of $\tilde{C}$ [$m^2$]")

Text(0.02, 0.5, 'Approximation terms of $\tilde{C}$ [$m^2$]')

# Load dataset

In [ ]:
import numpy as np
import scipy.interpolate
import xarray as xr
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from typing import Union, Literal, Callable, Tuple, List
import warnings

strength_cmap = sns.cubehelix_palette(start=0.5, rot=-0.5, as_cmap=True)

from sdm_eurec4a.visulization import (
    set_custom_rcParams,
    label_from_attrs,
    adjust_lightness_array,
    plot_one_one,
    handler_map_alpha,
    save_figure,
    add_second_x_axis,
)
from sdm_eurec4a.reductions import mean_and_stderror_of_mean
from sdm_eurec4a import RepositoryPath

from sdm_eurec4a import slurm_cluster
import dask.distributed

default_colors = set_custom_rcParams()
default_dark_colors = adjust_lightness_array(default_colors, 0.75)

RepoPaths = RepositoryPath("levante")

data_dir = RepoPaths.CLEO_data_dir / Path("output_v4.1")
fig_dir = RepoPaths.fig_dir / Path("paper-v4.1")
fig_dir.mkdir(exist_ok=True, parents=False)

In [ ]:
# try :
#     _, _ = client, cluster
# except Exception:
#     print("Starting dask cluster")
#     port = 8287
#     client, cluster = slurm_cluster.init_dask_slurm_cluster(
#         scale=1,
#         processes=16,
#         memory="32GB",
#         walltime="00:15:00",
#         scheduler_options={"dashboard_address": f":{port}"},
#     )
#     display(client)

NameError: name 'ds' is not defined

In [ ]:
def evaporation_mass2evaporation_energy(
    x: Union[xr.DataArray, np.ndarray]
) -> Union[xr.DataArray, np.ndarray]:
    L_v = 2.2257e3

    return x * L_v


def evaporation_mass2evaporation_precipitation(
    x: Union[xr.DataArray, np.ndarray]
) -> Union[xr.DataArray, np.ndarray]:
    rho_water = 1000  # kg / m^3
    # L_v = 2.2257e3

    return x * 3600 / 1000 * rho_water


def evaporation_precipitation2evaporation_mass(
    x: Union[xr.DataArray, np.ndarray]
) -> Union[xr.DataArray, np.ndarray]:
    rho_water = 1000  # kg / m^3
    L_v = 2257e3  # J / kg
    return x / 3600 * 1000 / rho_water


def evaporation_precipitation2evaporation_energy(
    x: Union[xr.DataArray, np.ndarray]
) -> Union[xr.DataArray, np.ndarray]:
    rho_water = 1000  # kg / m^3
    L_v = 2257e3  # J / kg

    # x in mm/h

    # output in W/m^2

    x = x / 1000 * rho_water / 3600  # kg/m^2/s

    return x * L_v  # W/m^2


def evap_rate_mass2evap_rate_energy(
    x: Union[xr.DataArray, np.ndarray]
) -> Union[xr.DataArray, np.ndarray]:

    L_v = 2257e3

    # x in g/m^3/s
    return x * L_v  # mW m^-3


def evap_rate_energy2evap_rate_mass(
    x: Union[xr.DataArray, np.ndarray]
) -> Union[xr.DataArray, np.ndarray]:

    L_v = 2257e3  # J / kg = W s / kg

    # x in mW m^-3
    return x / L_v  # g/m^3/s

In [ ]:
eulerian_data_path = lambda data_dir, microphysics: data_dir / Path(
    f"{microphysics}/combined/eulerian_dataset_combined.nc"
)
conservation_data_path = lambda data_dir, microphysics: data_dir / Path(
    f"{microphysics}/combined/conservation_dataset_combined.nc"
)


# mean_radius_data_path = lambda microphysics: data_dir / Path(
#     f"{microphysics}/combined/mean_radius_combined.nc"
# )

In [ ]:
microphysics = (
    "null_microphysics",
    "condensation",
    "collision_condensation",
    "coalbure_condensation_small",
    "coalbure_condensation_large",
)

compare_microphysics = (
    # "null_microphysics",
    "condensation",
    "collision_condensation",
    "coalbure_condensation_small",
    "coalbure_condensation_large",
)

labels = {
    "null_microphysics": "Null",
    "condensation": "Evap only",
    "collision_condensation": "Coll",
    "coalbure_condensation_small": "CoalBuRe few",
    "coalbure_condensation_large": "CoalBuRe many",
}

colors = {
    "null_microphysics": "grey",
    "condensation": "purple",
    "collision_condensation": "blue",
    "coalbure_condensation_small": "red",
    "coalbure_condensation_large": "orange",
}

dark_colors_list = adjust_lightness_array(colors.values(), 0.75)
dark_colors = dict([(key, str(dark_colors_list[i])) for i, key in enumerate(colors.keys())])

light_colors_list = adjust_lightness_array(colors.values(), 1.25)
light_colors = dict([(key, str(light_colors_list[i])) for i, key in enumerate(colors.keys())])

markers = {
    "null_microphysics": "$*$",
    "condensation": "1",
    "collision_condensation": "x",
    "coalbure_condensation_small": "+",
    "coalbure_condensation_large": "2",
}
dark_colors

{'null_microphysics': '#606060',
 'condensation': '#600060',
 'collision_condensation': '#0000bf',
 'coalbure_condensation_small': '#bf0000',
 'coalbure_condensation_large': '#bf7c00'}

## Data loading

In [ ]:
for mp in microphysics:
    print(f"Checking {mp}")
    try:
        xr.open_dataset(conservation_data_path(data_dir, mp))
    except ValueError:
        raise FileNotFoundError(f"Missing conservation {mp}")
    try:
        xr.open_dataset(eulerian_data_path(data_dir, mp))
    except ValueError:
        raise FileNotFoundError(f"Missing eulerian {mp}")

NameError: name 'microphysics' is not defined

In [ ]:
vars_to_use = [
    "air_temperature",
    "gridbox",
    "gridbox_bottom",
    "gridbox_coord3",
    "gridbox_top",
    "gridbox_volume",
    "gridbox_coord3_norm",
    "liquid_water_content",
    "mass_radius_mean",
    "mass_radius_std",
    "mass_represented_temporal_mean",
    "massdelta_condensation",
    "max_gridbox",
    "precipitation",
    "pressure",
    "radius_bins",
    "relative_humidity",
    "small_mass_radius_mean",
    "small_mass_radius_std",
    "small_xi_radius_mean",
    "small_xi_radius_std",
    "specific_mass_vapour",
    "surface_area",
    "xi_radius_mean",
    "xi_radius_std",
    "xi_temporal_mean",
]

### Merge of individual datasets

In [ ]:
time_slice = slice(1500, 3490)
# chunks = {'cloud_id' : 2}
chunks = {}

l = []
for microphysic in microphysics:
    print(microphysic)
    ds_euler = xr.open_dataset(eulerian_data_path(data_dir, microphysic))
    ds_euler = ds_euler[vars_to_use]
    ds_euler = ds_euler.sel(time=time_slice).mean("time", keep_attrs=True)
    ds_euler["surface_area"] = ds_euler["surface_area"].mean()
    ds_euler["surface_area"].attrs["units"] = "m^2"
    ds_euler["surface_area"].attrs["long_name"] = "Surface area"

    ds_euler["max_gridbox"] = ds_euler["max_gridbox"].fillna(ds_euler["gridbox"].max())

    # from kg per dT to g per h per m^2
    ds_conser = xr.open_dataset(conservation_data_path(data_dir, microphysic), chunks=chunks)
    ds_conser = ds_conser.sel(time=time_slice).mean("time", keep_attrs=True)

    # ds_mean_radius = xr.open_dataset(mean_radius_data_path(microphysic), chunks=chunks)

    ds_single = xr.merge([ds_conser, ds_euler])
    ds_single = ds_single.expand_dims(microphysics=(microphysic,))
    l.append(ds_single)
ds = xr.concat(l, dim="microphysics")
ds.chunk(chunks)

ds["max_gridbox"] = ds["max_gridbox"].fillna(ds["gridbox"].max())

ds["liquid_water_content"] = 1e3 * ds["liquid_water_content"]
ds["liquid_water_content"].attrs["units"] = "g/m^3"
ds["liquid_water_content"].attrs["long_name"] = "Rain water content"

ds["cloud_liquid_water_content"] = ds["liquid_water_content"].sel(gridbox=ds["max_gridbox"])
ds["cloud_liquid_water_content"].attrs["long_name"] = "Cloud rain water content"

# create variables for the mean radius of the distributions

# ds["cloud_xi_radius_mean"] = ds["xi_radius_mean"].sel(gridbox=ds["max_gridbox"])
# ds["cloud_xi_radius_mean"].attrs["long_name"] = "Cloud mean radius"
# ds["cloud_xi_radius_mean"].attrs["units"] = "µm"

# ds["small_cloud_xi_radius_mean"] = ds["small_xi_radius_mean"].sel(gridbox=ds["max_gridbox"])
# ds["small_cloud_xi_radius_mean"].attrs["long_name"] = "Cloud mean radius"
# ds["small_cloud_xi_radius_mean"].attrs["units"] = "µm"

ds["cloud_mass_radius_mean"] = ds["mass_radius_mean"].sel(gridbox=ds["max_gridbox"])
ds["cloud_mass_radius_mean"].attrs["long_name"] = "Cloud mean mass radius"
ds["cloud_mass_radius_mean"].attrs["units"] = "µm"

# ds["small_cloud_mass_radius_mean"] = ds["small_mass_radius_mean"].sel(gridbox=ds["max_gridbox"])
# ds["small_cloud_mass_radius_mean"].attrs["long_name"] = "Cloud mean mass radius"
# ds["small_cloud_mass_radius_mean"].attrs["units"] = "µm"

ds["source"].attrs["long_name"] = "Evaporation"
ds["inflow"].attrs["long_name"] = "Cloud Base Precipitation Flux"
ds["outflow"].attrs["long_name"] = "Surface Precipitation Flux"

# from kg/m^3/s

# # to mg/m^3/h
# ds['evaporation_rate']  = - ds['massdelta_condensation'] * 1e6 * 3600
# ds['evaporation_rate'].attrs['units'] = 'mg/m^3/h'
# ds['evaporation_rate'].attrs['long_name'] = 'Evaporation rate'

# to mm/m/h
rho_water = 1000  # kg / m^3
ds["evaporation_rate"] = -1e3 / rho_water * ds["massdelta_condensation"] * 3600
ds["evaporation_rate"].attrs["units"] = r"mm \, h^{-1} \, m^{-1}"
ds["evaporation_rate"].attrs["long_name"] = "Evaporation rate"

# from kg / m^3 / s
# to W / m^3
vapourization_heat = 2257e3  # J / kg
rho_water = 1000  # kg / m^3
ds["evaporation_rate_energy"] = ds["massdelta_condensation"] * vapourization_heat  # J/s / m^3 = W / m^3
ds["evaporation_rate_energy"] = 1e3 * ds["evaporation_rate_energy"]  # mW / m^3
ds["evaporation_rate_energy"].attrs["units"] = r"mW \, m^{-3}"
ds["evaporation_rate_energy"].attrs["long_name"] = "Evaporation rate"


for var, new_var in zip(
    ["source", "inflow", "outflow", "reservoir_change"],
    [
        "source_precipitation",
        "inflow_precipitation",
        "outflow_precipitation",
        "reservoir_change_precipitation",
    ],
):
    attrs = ds[var].attrs.copy()
    # from  kg per dT per domain area
    # dT = 2s

    # # to    g per h per m^2
    # ds[var] = ds[var] / 2 * 3600 / ds['surface_area'] * 1e6
    # ds[var].attrs.update(attrs)
    # ds[var].attrs['units'] = 'mg/m^2/h'

    # to    mm / h
    rho_water = 1000  # kg / m^3
    ds[new_var] = ds[var] / ds["time"].diff("time") * 3600 / ds["surface_area"]  # kg / m^2 / h
    ds[new_var] = 1e3 * ds[new_var] / rho_water  # mm / h
    ds[new_var].attrs.update(attrs)
    ds[new_var].attrs["units"] = r"mm \, h^{-1}"

for var, new_var in zip(
    ["source", "inflow", "outflow", "reservoir_change"],
    ["source_energy", "inflow_energy", "outflow_energy", "reservoir_change_energy"],
):
    attrs = ds[var].attrs.copy()
    # from  kg per dT per domain area
    # dT = 2s

    # # to    g per h per m^2
    # ds[var] = ds[var] / 2 * 3600 / ds['surface_area'] * 1e6
    # ds[var].attrs.update(attrs)
    # ds[var].attrs['units'] = 'mg/m^2/h'

    # to    mm / h
    vapourization_heat = 2257e3  # J / kg
    rho_water = 1000  # kg / m^3
    ds[new_var] = ds[var] / ds["time"].diff("time") / ds["surface_area"]  # kg / m^2 / s
    ds[new_var] = ds[new_var] * vapourization_heat  # J/s = W / m^2
    ds[new_var].attrs.update(attrs)
    ds[new_var].attrs["units"] = r"W \, m^{-2}"


ds["evaporation_fraction"] = -100 * ds["source"] / ds["inflow"]
ds["evaporation_fraction"].attrs["units"] = "\\%"
ds["evaporation_fraction"].attrs["long_name"] = "Evaporation fraction"

sorted_cloud_id = ds["cloud_id"].sortby(ds["evaporation_rate"].max(["gridbox", "microphysics"]))
ds = ds.load()
ds = ds.sel(cloud_id=sorted_cloud_id[:-1])
# ds_diff = ds - ds.sel(microphysics = 'condensation')

null_microphysics
condensation
collision_condensation
coalbure_condensation_small
coalbure_condensation_large


In [ ]:
ds

In [ ]:
ds["air_temperature"].attrs.update(long_name="Air temperature")
ds["pressure"].attrs.update(long_name="Pressure")
ds["specific_mass_vapour"].attrs.update(long_name="Specific mass vapour")

for var in [
    "mass_radius_mean",
    "mass_radius_std",
    "xi_radius_mean",
    "xi_radius_std",
]:
    ds[var].attrs.update(units="µm")

In [ ]:
# ds = ds.sel(cloud_id=sorted_cloud_id[:-2])
ds = ds.sortby("cloud_id")
ds_diff = ds - ds.sel(microphysics="condensation")

for var in ds_diff.data_vars:
    ds_diff[var].attrs.update(ds[var].attrs)
    try:
        ds_diff[var].attrs["long_name"] = f"Difference {ds[var].attrs['long_name']}"
    except KeyError:
        print(f"Missing long_name for {var}")

ds_relative_diff = (ds / ds.sel(microphysics="condensation") - 1) * 100

for var in ds_diff.data_vars:
    ds_relative_diff[var].attrs.update(ds[var].attrs)
    try:
        ds_relative_diff[var].attrs["long_name"] = f"Relative difference {ds[var].attrs['long_name']}"
        ds_relative_diff[var].attrs["units"] = r"\%"
    except KeyError:
        print(f"Missing long_name for {var}")

Missing long_name for mass_represented_temporal_mean
Missing long_name for xi_temporal_mean
Missing long_name for mass_represented_temporal_mean
Missing long_name for xi_temporal_mean


# Apply to the data

In [ ]:
height = 800

T = ds["air_temperature"].sel(microphysics="condensation")
p = ds["pressure"].sel(microphysics="condensation")
RH = 0.01 * ds["relative_humidity"].sel(microphysics="condensation")
r = ds["radius_bins"] * 1e-6
mmr = ds["cloud_mass_radius_mean"].sel(microphysics="condensation") * 1e-6

In [ ]:
(RepoPaths.data_dir / "sharing" / "florian").mkdir(exist_ok=True, parents=True)

In [ ]:
ds_florian = ds[
    [
        "gridbox_coord3",
        "max_gridbox",
        "gridbox_volume",
        "air_temperature",
        "pressure",
        "relative_humidity",
        "specific_mass_vapour",
        "evaporation_rate_energy",
        "evaporation_fraction",
        "cloud_mass_radius_mean",
    ]
]
ds_florian = ds_florian.sel(microphysics="condensation")
ds_florian_vertical = ds_florian.copy().drop_vars(["gridbox_coord3", "max_gridbox", "gridbox_volume"])
for var in [
    "air_temperature",
    "pressure",
    "relative_humidity",
    "specific_mass_vapour",
    "evaporation_rate_energy",
]:
    attrs = ds_florian[var].attrs
    ds_florian_vertical[var] = (
        (ds_florian[var] * ds_florian["gridbox_volume"]) / ds_florian["gridbox_volume"].sum("gridbox")
    ).sum("gridbox")
    ds_florian_vertical[var].attrs.update(attrs)
    ds_florian_vertical[var].attrs["long_name"] = f"Vertical mean {attrs['long_name']}"

ds_florian.to_netcdf(
    RepoPaths.data_dir / "sharing" / "florian" / "cleo_output_evaporation_only_setup.nc"
)
ds_florian_vertical.to_netcdf(
    RepoPaths.data_dir / "sharing" / "florian" / "veritcal_average_cleo_output_evaporation_only_setup.nc"
)
ds_florian_vertical

In [ ]:
cloud_id = np.random.choice(ds["cloud_id"], 1)
var = "evaporation_rate_energy"
plt.scatter(
    ds_florian[var].sel(cloud_id=cloud_id).T,
    ds_florian["gridbox_coord3"].sel(cloud_id=cloud_id).T,
)
plt.axvline(ds_florian_vertical[var].sel(cloud_id=cloud_id), color="red")
plt.axvline(ds_florian[var].sel(cloud_id=cloud_id).mean("gridbox"), color="blue")

In [ ]:
ds_sel = ds[
    [
        "mass_radius_mean",
        "mass_radius_std",
        "xi_radius_mean",
        "xi_radius_std",
        "evaporation_fraction",
        "max_gridbox",
    ]
].sel(microphysics="condensation")
ds_sel = ds_sel.sel(gridbox=ds_sel["max_gridbox"])

cmap = sns.color_palette("rocket_r", as_cmap=True)


fig, axs = plt.subplots(ncols=2, sharey=True, sharex=True)
x = ds_sel["mass_radius_mean"]
y = ds_sel["evaporation_fraction"]
c = ds_sel["mass_radius_std"]
norm = mcolors.Normalize(vmin=c.min(), vmax=c.max())


sc = axs[0].scatter(
    x,
    y,
    c=c,
    cmap=cmap,
    norm=norm,
)
axs[0].set_xscale("log")
axs[0].set_yscale("log")
fig.colorbar(mappable=sc, ax=axs[0], label=label_from_attrs(c), orientation="horizontal")
axs[0].set_xlabel(label_from_attrs(x))
axs[0].set_ylabel(label_from_attrs(y))

axs[0].set_title("Volume weighted radius")

x = ds_sel["xi_radius_mean"]
y = ds_sel["evaporation_fraction"]
c = ds_sel["xi_radius_std"]
norm = mcolors.Normalize(vmin=c.min(), vmax=c.max())


sc = axs[1].scatter(
    x,
    y,
    c=c,
    cmap=cmap,
    norm=norm,
)
axs[1].set_xscale("log")
axs[1].set_yscale("log")
fig.colorbar(mappable=sc, ax=axs[1], label=label_from_attrs(c), orientation="horizontal")
axs[1].set_xlabel(label_from_attrs(x))
axs[1].set_ylabel(label_from_attrs(y))

axs[1].set_title("Radius")

Text(0.5, 1.0, 'Radius')

In [ ]:
td = Thermodyanmics(T0=273.15, p0=101325)
c_all = td.constant_nils(T=T, p=p, relative_humidity=RH)
c_mean = td.constant_nils(T=T.mean("gridbox"), p=p.mean("gridbox"), relative_humidity=RH.mean("gridbox"))
c_florian = td.constant_florian(relative_humidity=RH.mean("gridbox"))

bins = dict(
    temperature=np.arange(285, 305, 1),
    pressure=np.arange(900, 1100, 10),
    relative_humidity=np.arange(0, 1, 0.01),
)

r2 = r**2

fig, axs = plt.subplots(ncols=5, figsize=(20, 5))
bins = np.arange(900, 1100, 10)
axs[0].hist(p.values.flatten() / 100, density=True, bins=bins)
axs[0].hist(p.mean("gridbox").values.flatten() / 100, density=True, bins=bins)

bins = np.arange(285, 305, 1)
axs[1].hist(T.values.flatten(), density=True, bins=bins)
axs[1].hist(T.mean("gridbox").values.flatten(), density=True, bins=bins)

bins = np.arange(0.7, 1.02, 0.02)
axs[2].hist(RH.values.flatten(), density=True, bins=bins)
axs[2].hist(RH.mean("gridbox").values.flatten(), density=True, bins=bins)

bins = np.geomspace(1e-11, 1e-5, 100)
axs[3].hist((r**2), density=False, bins=bins)
axs[3].hist((mmr**2), density=False, bins=bins)
axs[3].set_xscale("log")

bins = np.geomspace(1e-12, 3e-5, 40)
# axs[4].hist(c.values.flatten(), density=True, bins = bins)
axs[4].hist(c_mean.values.flatten(), density=False, bins=bins)
axs[4].hist(c_florian.values.flatten(), density=False, bins=bins)
axs[4].set_xscale("log")

In [ ]:
result_all = td.evapoartion_fraction_func(
    radius=ds["radius_bins"] * 1e-6,
    height=height,
    method="nils",
    kwargs={"T": T, "p": p, "relative_humidity": RH},
)

c_all = td.constant_nils(T=T, p=p, relative_humidity=RH)
result_gridbox = td.evapoartion_fraction_func(
    radius=ds["radius_bins"] * 1e-6,
    height=height,
    method="nils",
    kwargs={"T": T.mean("gridbox"), "p": p.mean("gridbox"), "relative_humidity": RH.mean("gridbox")},
)
c_gridbox = td.constant_nils(
    T=T.mean("gridbox"), p=p.mean("gridbox"), relative_humidity=RH.mean("gridbox")
)
result_mean = td.evapoartion_fraction_func(
    radius=ds["radius_bins"] * 1e-6,
    height=height,
    method="nils",
    kwargs={"T": T.mean(), "p": p.mean(), "relative_humidity": RH.mean()},
)
c_mean = td.constant_nils(T=T.mean(), p=p.mean(), relative_humidity=RH.mean())

In [ ]:
c.min(), c.max()

(np.float64(0.05), np.float64(0.05))

In [ ]:
fig, ax = plt.subplots()

x = ds["radius_bins"]
y = ds["gridbox_coord3_norm"].sel(microphysics="condensation").transpose("gridbox", ...)
c = -100 * result_all.transpose("radius_bins", "gridbox", ...)

all_dims = set(x.dims).union(y.dims).union(c.dims)
print(all_dims)
x = x.expand_dims({dim: ds[dim] for dim in all_dims - set(x.dims)})
y = y.expand_dims({dim: ds[dim] for dim in all_dims - set(y.dims)})
c = c.expand_dims({dim: ds[dim] for dim in all_dims - set(c.dims)})

x = x.isel(cloud_id=20)
y = y.isel(cloud_id=20)
c = c.isel(cloud_id=20)

x = x.transpose("radius_bins", "gridbox", ...)  # .values.reshape(x.shape[0], -1)
y = y.transpose("radius_bins", "gridbox", ...)  # .values.reshape(x.shape[0], -1)
c = c.transpose("radius_bins", "gridbox", ...)  # .values.reshape(x.shape[0], -1)

cmap = sns.color_palette("rocket_r", as_cmap=True)
norm = mcolors.LogNorm(vmin=1e-5, vmax=c.max())

sc = ax.scatter(x, y, c=c, cmap=cmap, norm=norm)
# fig.colorbar(sc, ax=ax)

# ax.set_ylim(-1e3, 1e3)
ax.set_xscale("log")
# ax.set_yscale('symlog', linthresh=1e-4, linscale = 0.1)

{'cloud_id', 'radius_bins', 'gridbox'}


In [ ]:
mp = "condensation"

fig, ax = plt.subplots()

x = ds["cloud_mass_radius_mean"].sel(microphysics=mp)
y = ds["evaporation_fraction"].sel(microphysics=mp)

# result_x = f(ds['radius_bins'] * 1e-6, c.mean("cloud_id"))


ax.scatter(
    x,
    y,
    c=colors[mp],
    label=f"Simulations\n(x-axis mean mass radius)",
    marker=markers[mp],
)

ax.set_xscale("log")
ax.set_yscale("log")

# ax.plot(
#     ds['radius_bins'],
#     - 100 * result_gridbox.T,
#     linewidth = 0.3,
#     color = 'green',
#     alpha = 0.3,
# );
# ax.plot(
#     np.nan,
#     np.nan,
#     linewidth = 0.3,
#     color = 'green',
#     alpha = 1,
#     label = n"Approximation\Cloud speci\nBased on vertical mean T, p, RH",
# )

ax.plot(
    ds["radius_bins"],
    -100 * result_mean,
    label="Exact function using mean T, p, RH",
    color="k",
)

approximation = (
    100
    * 3
    / 2
    * c_mean
    * height
    / td.terminal_velocity(ds["radius_bins"] * 1e-6)
    * (ds["radius_bins"] * 1e-6) ** (-2)
)
ax.plot(
    ds["radius_bins"],
    approximation.where(approximation < 100),
    label="$3/2 \\. C r^{-2}$ approximation",
    color="orange",
    linestyle="--",
)

T_mean = T.min()
p_mean = p.min()
v_mean = 1

# ax.plot(
#     ds['radius_bins'],
#     100 * ( 3/2 * c_mean * (1e-6 * ds['radius_bins']) ** (-2)).T,
#     label = "approximation r^-2",
#     color = 'blue',
# );
plt.legend(loc="lower left")
# plt.ylim(1e-1, 1.1e2)
# plt.xlim(6e1, 2e3)
plt.ylabel("Evaporation fraction in %")
plt.xlabel("Radius in µm")

Text(0.5, 0, 'Radius in µm')

# Store data as txt for Florian

In [ ]:
mp = "condensation"
x = ds["cloud_mass_radius_mean"].sel(microphysics=mp)
y = ds["evaporation_fraction"].sel(microphysics=mp)

df = xr.merge([x, y]).drop_vars(["microphysics"]).to_dataframe()
df.to_csv("evaporation_fraction.csv")
df.to_json("evaporation_fraction.json")
# store as txt file
df.to_csv("evaporation_fraction.txt", header=False, index=False)

# .to_csv("evaporation_fraction.csv")

plt.scatter(x, y)

In [ ]:
plt.pcolormesh(
    result_gridbox["radius_bins"],
    np.arange(len(result_gridbox["cloud_id"])),
    result_gridbox,
)
plt.xscale("log")
plt.xlim(1e2, 3e3)

(100.0, 3000.0)

In [ ]:
d = {"T": T.mean("gridbox"), "p": p.mean("gridbox"), "relative_humidity": RH.mean("gridbox")}


fig, axs = plt.subplots(ncols=3, figsize=(10, 5))

for d, ax in zip(d.values(), axs):
    ax.plot(
        d,
    )